In [101]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.market.curves import *
import datetime as dt
import numpy as np

In [102]:
 """
Calculating the Duration and Other Interest Rate Sensitivity Metrics in an Interest Rate Swap
Based on Bond Math: The Theory Behind the Formulas, Second Edition by Donald J. Smith
 """

'\nCalculating the Duration and Other Interest Rate Sensitivity Metrics in an Interest Rate Swap\nBased on Bond Math: The Theory Behind the Formulas, Second Edition by Donald J. Smith\n'

# 1.Interest rate swaps at the beginning

## Define the swap

In [103]:
swap_cal_type = CalendarTypes.TARGET
bd_type = BusDayAdjustTypes.FOLLOWING
dg_type = DateGenRuleTypes.BACKWARD

fixed_cpn = 0.034
fixed_freq_type = FrequencyTypes.QUARTERLY
fixed_dc_type = DayCountTypes.THIRTY_360_BOND

float_spread = 0.0
float_freq_type = FrequencyTypes.QUARTERLY
float_dc_type = DayCountTypes.THIRTY_360_BOND

swap_type = SwapTypes.PAY
notional = 60000000

start_dt = Date(1, 1, 2024)
maturity_dt = start_dt.add_tenor('2Y')

swap = IborSwap(start_dt,
                maturity_dt,
                swap_type,
                fixed_cpn,
                fixed_freq_type,
                fixed_dc_type,
                notional,
                float_spread,
                float_freq_type,
                float_dc_type,
                swap_cal_type,
                bd_type,
                dg_type)


In [104]:
value_date = dt.datetime(2023, 12, 31)
value_dt = from_datetime(value_date)
settle_dt = value_dt.add_weekdays(0)


## Define the yield curve

In [105]:
yield_curve = [i / 100 for i in [0.5, 1.0407, 1.5829, 2.1271, 2.4506, 2.7756, 3.1025, 3.4316]]
zero_dts = value_dt.add_years([0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0])

zero_curve = DiscountCurveZeros(value_dt=value_dt,
                                zero_dts=zero_dts,
                                zero_rates=yield_curve,
                                freq_type=FrequencyTypes.QUARTERLY,
                                dc_type=DayCountTypes.THIRTY_360_BOND,
                                interp_type=InterpTypes.FINCUBIC_ZERO_RATES
                                )

## Calculate the swap rate and sensitivity indicators

In [106]:
swap_rate = swap.swap_rate(value_dt, zero_curve)
swap_rate

0.03404344249149676

In [107]:
swap.value(settle_dt, zero_curve)

5070.7014320408925

In [108]:
payment_periods = 8
payer_side_macaulay_duration = swap.payer_side_macaulay_duration(value_dt, zero_curve, payment_periods)
payer_side_macaulay_duration

-6.767653343606554

In [109]:
receiver_side_macaulay_duration = swap.receiver_side_macaulay_duration(value_dt, zero_curve, payment_periods)
receiver_side_macaulay_duration

6.767653343606554

In [110]:
payer_side_modified_duration = swap.payer_side_modified_duration(value_dt, zero_curve, payment_periods)
payer_side_modified_duration

-6.710540865595371

In [111]:
receiver_side_modified_duration = swap.receiver_side_modified_duration(value_dt, zero_curve, payment_periods)
receiver_side_modified_duration

6.710540865595371

In [112]:
payer_side_BPV = swap.payer_side_BPV(value_dt, zero_curve, payment_periods)
payer_side_BPV

-10065.811298393059

In [113]:
receiver_side_BPV = swap.receiver_side_BPV(value_dt, zero_curve, payment_periods)
receiver_side_BPV

10065.811298393059

In [114]:
index = ['swap_rate',
         'payer_side_macaulay_duration',
         'payer_side_modified_duration',
         'payer_side_BPV',
         'receiver_side_macaulay_duration',
         'receiver_side_modified_duration',
         'receiver_side_BPV']
data_financepy = [swap_rate,
                  payer_side_macaulay_duration, payer_side_modified_duration, payer_side_BPV,
                  receiver_side_macaulay_duration, receiver_side_modified_duration, receiver_side_BPV]
data_bondmath = [0.034,
                 np.nan, np.nan, np.nan,
                 np.nan, np.nan, np.nan]
data = {
    'financepy': data_financepy,
    'Bond Math by Donald J. Smith': data_bondmath
}

df = pd.DataFrame(data, index=index)
df

,financepy,Bond Math by Donald J. Smith
swap_rate,0.034043,0.034
payer_side_macaulay_duration,-6.767653,NaN
payer_side_modified_duration,-6.710541,NaN
payer_side_BPV,-10065.811298,NaN
receiver_side_macaulay_duration,6.767653,NaN
receiver_side_modified_duration,6.710541,NaN
receiver_side_BPV,10065.811298,NaN


# 2.Calculation of indicators after 3 months

## update the swap

In [115]:
fixed_cpn_3m = 0.034
start_dt_3m = Date(1, 4, 2024)
maturity_dt_3m = start_dt_3m.add_years(1.75)

swap_3m = IborSwap(start_dt_3m,
                   maturity_dt_3m,
                   swap_type,
                   fixed_cpn_3m,
                   fixed_freq_type,
                   fixed_dc_type,
                   notional,
                   float_spread,
                   float_freq_type,
                   float_dc_type,
                   swap_cal_type,
                   bd_type,
                   dg_type)



In [116]:
value_date_3m = dt.datetime(2024, 3, 31)
value_dt_3m = from_datetime(value_date_3m)

settle_dt_3m = value_dt_3m.add_weekdays(0)
settle_dt_3m

31-MAR-2024

## update the yield curve

In [117]:
yield_curve_3m = [i / 100 for i in [0.75, 1.1853, 1.6822, 2.1474, 2.4371, 2.7390, 3.0198]]
zero_dts_3m = value_dt_3m.add_years([0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75])

zero_curve_3m = DiscountCurveZeros(value_dt_3m,
                                   zero_dts=zero_dts_3m,
                                   zero_rates=yield_curve_3m,
                                   freq_type=FrequencyTypes.QUARTERLY,
                                   dc_type=DayCountTypes.THIRTY_360_BOND,
                                   interp_type=InterpTypes.FINCUBIC_ZERO_RATES
                                   )



In [118]:
swap_3m.value(value_dt_3m, zero_curve_3m)

-404094.769553849

In [119]:
data_financepy_3m = [swap_3m.swap_rate(value_dt_3m, zero_curve_3m),
                     swap_3m.payer_side_macaulay_duration(value_dt_3m, zero_curve_3m, payment_periods),
                     swap_3m.payer_side_modified_duration(value_dt_3m, zero_curve_3m, payment_periods),
                     swap_3m.payer_side_BPV(value_dt_3m, zero_curve_3m, payment_periods),
                     swap_3m.receiver_side_macaulay_duration(value_dt_3m, zero_curve_3m, payment_periods),
                     swap_3m.receiver_side_modified_duration(value_dt_3m, zero_curve_3m, payment_periods),
                     swap_3m.receiver_side_BPV(value_dt_3m, zero_curve_3m, payment_periods)]

data_bondmath_3m = [0.030,
                    -6.769,
                    -6.7186,
                    -10077.9,
                    6.769,
                    6.7186,
                    10077.9]

data_3m = {
    'financepy': data_financepy_3m,
    'Bond Math by Donald J. Smith': data_bondmath_3m
}

df3m = pd.DataFrame(data_3m, index=index)
df3m

,financepy,Bond Math by Donald J. Smith
swap_rate,0.030066,0.0300
payer_side_macaulay_duration,-6.794199,-6.7690
payer_side_modified_duration,-6.743512,-6.7186
payer_side_BPV,-10115.267497,-10077.9000
receiver_side_macaulay_duration,6.794199,6.7690
receiver_side_modified_duration,6.743512,6.7186
receiver_side_BPV,10115.267497,10077.9000
